In [ ]:
import os
import pickle
import json

import numpy as np
import matplotlib.pyplot as plt

# import rose packages
from rose.optimisation.optimisation import Optimisation, OptimisationModelPart, ModelResults
from SignalProcessing.signal_tools import Signal
from data_proc import ricardo, SoS


# Optimisation
Note that, to use this notebook, the notebook "rose_demo" has to be completed

# Read SoS

In [ ]:
sos_dir = "../data/SoS"
sos = SoS.ReadSosScenarios(os.path.join(sos_dir, "soilprofiles.csv"),
                               os.path.join(sos_dir, "20201102_Prorail_parameters_SOS.csv"),
                               os.path.join(sos_dir, "segments.csv"),
                               os.path.join(sos_dir, "Segments_TKI_v2.shp"))
sos.create_segments()
sos.dump(os.path.join(sos_dir, "SOS.json"))

# Load SoS

In [ ]:
with open(os.path.join(sos_dir, "SOS.json"), 'r') as f:
    sos_data = json.load(f)

# Read Ricardo data

In [ ]:
ricardo_filenames = [r"../data/Ricardo/Jan.json",
             r"../data/Ricardo/Jun.json",
             ]
ricardo_output_fn = r"../data/Ricardo/inframon.pickle"
ricardo.read_inframon(ricardo_filenames, ricardo_output_fn)

# Load Ricardo data

In [ ]:
ricardo_data = ricardo.load_inframon_data(ricardo_output_fn)

# Get SoS Coordinates


In [ ]:
# get coordinates of a segment
segment = sos_data["Segment 1030"]
coordinates = np.array(list(segment.values())[0]['coordinates'])

# get coordinate limits
xlim = [min(coordinates[:,0]), max(coordinates[:,0])]
ylim = [min(coordinates[:,1]), max(coordinates[:,1])]

# Get Ricardo data within SoS limits

In [ ]:
ricardo_data_within_bounds = ricardo.get_data_within_bounds(ricardo_data["Jan"], xlim, ylim)

# Check if data is valid
The train velocity should be relatively constant over the bounded track, such that the acceleration signal is representative for a certain train speed at that point of the track

## check train velocity

In [ ]:
ricardo.plot_train_velocity(ricardo_data_within_bounds)
plt.grid()

## check acceleration signal

In [ ]:
ricardo.plot_acceleration_signal(ricardo_data_within_bounds["time"], ricardo_data_within_bounds["acc_side_1"])
plt.grid()

# Filter inframon signal

In [ ]:
# filter Ricardo measurements

# set filter settings
settings_filter = {"FS": 250,                   # acquisition frequency
                   "cut-off_high": 120,         # high cutoff frequency
                   "cut-off_low": 40,           # low cutoff frequency 
                   "n": 10,                     # order of the filter
                   "smoothing_distance": 10,    # distance of smoothed wavelength
                   }    

acceleration_data = ricardo_data_within_bounds["acc_side_1"]

# set ricardo signal
ricardo_signal = Signal(ricardo_data_within_bounds["time"],acceleration_data, settings_filter["FS"])


# filter high frequencies
ricardo_signal.filter(settings_filter["cut-off_high"], settings_filter["n"], type_filter="lowpass")

# filter low frequencies
ricardo_signal.filter(settings_filter["cut-off_low"], settings_filter["n"], type_filter="highpass")

# copy filtered signal
acc = np.copy(ricardo_signal.signal)


In [ ]:
# plot filtered signal
ricardo.plot_acceleration_signal(ricardo_data_within_bounds["time"], acc)
plt.grid()

# Integrate acceleration signal to velocity signal



In [ ]:
ricardo_signal.integrate(hp=True, moving=True, baseline=False, ini_cond=0)

In [ ]:
ricardo.plot_velocity_signal(ricardo_data_within_bounds["time"], ricardo_signal.signal)
plt.grid()

# Fast fourier transformation velocity signal

In [ ]:
# fast fourier transformation of velocity signal

ricardo_signal.fft(half_representation=True)
freq_velocity = ricardo_signal.frequency
ampl_velocity = ricardo_signal.amplitude

# smooth signal
ampl_velocity = ricardo.smooth_signal_within_bounds_over_wave_length(ricardo_data_within_bounds, settings_filter["smoothing_distance"], ampl_velocity)

In [ ]:
ricardo.plot_fft_velocity_signal(ricardo_data_within_bounds,acc, settings_filter["smoothing_distance"])
plt.grid()

In [ ]:
# Get maximum velocity amplitude and corresponding frequency

m_to_mm = 1000

# maximum amplitude
max_vel_ampl = ampl_velocity[np.argmax(ampl_velocity)] * m_to_mm
# corresponding frequency
freq_at_max_vel = freq_velocity[np.argmax(ampl_velocity)]

In [ ]:
max_vel_ampl, freq_at_max_vel 

# Check if frequency corresponds to the train speed
In this example, sleepers have a distance from eachother of 0.6 m. From the ricardo data, we can see that the average train speed over the segment is approximately 128 km/h. The frequency at the highest signal velocity should correspond to this train speed.

In [ ]:
train_speed_mps = 128/3.6         # train speed in m/s
sleeper_dist = 0.6                # sleeper distance

load_frequency = train_speed_mps/sleeper_dist

In [ ]:
load_frequency

# Update Rose model

In [ ]:
# load results from rose demo

output_dir = "res"

with open(os.path.join(output_dir, "all_results.pickle"), 'rb') as f:
    coupled_model = pickle.load(f)["coupled_model"]

In [ ]:
coupled_model.train.velocities

## reset velocities


In [ ]:
# set velocity of train
velocities = np.ones(len(coupled_model.time)) * train_speed_mps

# prevent train from moving in initialisation phase
velocities[0:len(coupled_model.initialisation_time)] = 0
coupled_model.train.velocities = velocities

In [ ]:
# set more robust solver
from solvers.newmark_solver import NewmarkImplicitForce
solver = NewmarkImplicitForce()
coupled_model.solver = solver

# Set optimisation parameters

In [ ]:
optimisation = Optimisation()
optimisation.model=coupled_model
optimisation.observations =[freq_at_max_vel, max_vel_ampl]


optimisation_model_part = OptimisationModelPart()
optimisation_model_part.model_part = coupled_model.track.model_parts[4] # soil

# set the parameter in the corresponding model part which should be optimised
optimisation_model_part.optimisation_parameter_names = ["stiffness"]

# Set model result settings

## create a function to perform a fast fourier transformation on rose output

In [ ]:
def results_function(signal, aq_freq):
    """
    Results function to be performed after a coupled calculation
    
    This functoin takes a signal, performs a fast fourier transformation and returns 
    the maximum amplitude in the frequency domain and the corresponding frequency 

    :param signal: signal to be processed
    :param aq_freq: aquisition frequency
    :return:
    """
    from SignalProcessing.signal_tools import Signal
    import matplotlib.pyplot as plt
    
    # set signal
    sig = Signal(0,signal[:,0], aq_freq[0])
    
    # filter signal
    sig.filter(40, 6, type_filter="highpass")

    # fast fourier transformation
    sig.fft(half_representation=True)
    freq = sig.frequency
    ampl = sig.amplitude

    # get maximum amplitude and corresponding frequency
    max_ampl_idx = np.argmax(ampl)
    max_freq = freq[max_ampl_idx]
    max_ampl = ampl[max_ampl_idx]
    return max_freq, max_ampl

## the results to be transformed are the velocities in the front wheel of the train

In [ ]:
coupled_model.velocities_out

## get the vertical displacement degree of freedom index of the front wheel of the train

In [ ]:
result_index = coupled_model.train.wheels[-1].nodes[0].index_dof[1]

In [ ]:
coupled_model.velocities_out[result_index,:]

In [ ]:
model_results = ModelResults()

# set name of result attribute as present in the coupled model
model_results.result_names = ["velocities_out"]
# set the index of the result to be used
model_results.result_indices = [result_index]
# set the timesteps to be used for the optimisation
model_results.time_step_indices = np.arange(int(len(coupled_model.initialisation_time)),int(len(coupled_model.time)))
# set result function
model_results.result_function = results_function

In [ ]:
# calculate aquisition frequency
dt = np.diff(coupled_model.time)[-1]

aq_freq = 1/dt

# set model results arguments
model_results.args =[aq_freq]

## Initialise optimisation algorithm
The optimisation algorithm which is used is the levenberg marquart algorithm. Which is a gradient based algorithm which works efficiently when only a few parameters are to be optimised and the problem is not highly non-linear.

In [ ]:
optimisation.model_results = [model_results]
optimisation.optimisation_model_parts = [optimisation_model_part]
optimisation.initialise()

## perform optimisation

In [ ]:
initial_guess_stiffness = 5e7

In [ ]:
optimisation.least_square(np.array([initial_guess_stiffness]), ftol=1e-8)